In [86]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

In [87]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

torch.manual_seed(125)
if torch.cuda.is_available() : 
    torch.cuda.manual_seed_all(125)
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (1.0,))
])

In [88]:
from torchvision.datasets import MNIST

download_root = '../080289-main/chap07/MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, 
                      train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform,
                     train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform,
                    train=False, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../080289-main/chap07/MNIST_DATASET\MNIST\raw\train-images-idx3-ubyte.gz to ../080289-main/chap07/MNIST_DATASET\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../080289-main/chap07/MNIST_DATASET\MNIST\raw\train-labels-idx1-ubyte.gz to ../080289-main/chap07/MNIST_DATASET\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../080289-main/chap07/MNIST_DATASET\MNIST\raw\t10k-images-idx3-ubyte.gz to ../080289-main/chap07/MNIST_DATASET\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../080289-main/chap07/MNIST_DATASET\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../080289-main/chap07/MNIST_DATASET\MNIST\raw



In [89]:
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [90]:
batch_size=100
n_iters=6000
num_epochs=n_iters/(len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

## LSTM CELL 

In [91]:
class LSTMCell(nn.Module) :
    def __init__(self, input_size, hidden_size, bias=True) :
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4*hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4*hidden_size, bias=bias)
        self.reset_parameters()
        
    def reset_parameters(self) :
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters() :
            w.data.uniform_(-std, std)
            
    def forward(self, x, hidden) :
        hx, cx = hidden
        x = x.view(-1, x.size(1))
        
        gates = self.x2h(x) + self.h2h(hx)
        gates = gates.squeeze()
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)
        
        ingate = F.sigmoid(ingate) # 입력 게이트에 시그모이드 적용
        forgetgate = F.sigmoid(forgetgate) # 망각 게이트에 시그모이드 적용
        cellgate = F.tanh(cellgate) # 셀 게이트에 탄젠트 적용
        outgate = F.sigmoid(outgate) # 출력 게이트에 시그모이드 적용
        
        cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
        hy = torch.mul(outgate, F.tanh(cy))
        
        return (hy, cy)

## LSTM cell 네트워크

In [92]:
class LSTMModel(nn.Module) :
   def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True) :
       super(LSTMModel, self).__init__()
       self.hidden_dim = hidden_dim
       
       self.layer_dim = layer_dim
       self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)
       self.fc = nn.Linear(hidden_dim, output_dim)
       
   def forward(self, x) :
       if torch.cuda.is_available() :
           h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
       else :
           h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
           
       if torch.cuda.is_available() :
           c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
       else :
           c0 = Variable(torch.zeros(self.layer_dim, x.size(0), hidden_dim))
           
       outs = []
       cn =  c0[0,:,:]
       hn = h0[0,:,:]
       
       for seq in range(x.size(1)) :
           hn, cn = self.lstm(x[:, seq, :], (hn, cn))
           outs.append(hn)
           
       out = outs[-1].squeeze()
       out = self.fc(out)
       return out

## LSTM 네트워크 구성

In [93]:
input_dim=28
hidden_dim=128
layer_dim=1
output_dim=18

model =  LSTMModel(input_dim, hidden_dim, layer_dim, output_dim) 
if torch.cuda.is_available() :
   model.cuda()
   
criterion = nn.CrossEntropyLoss()
learning_rate =0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 최적화 및 손실함수 정의

In [94]:
seq_dim=28
loss_list = []
iter = 0
for epoch in range(num_epochs) :
   for i, (images ,labels) in enumerate(train_loader) :
       if torch.cuda.is_available() :
           images = Variable(images.view(-1, seq_dim, input_dim).cuda())
           labels = Variable(labels.cuda())
       
       else :
           images = Variable(images.view(-1, seq_dim, input_dim))
           labels = Variable(labels)
           
       optimizer.zero_grad()
       outputs = model(images)
       loss = criterion(outputs, labels)
       
       if torch.cuda.is_available() :
           loss.cuda()
           
       loss.backward()
       optimizer.step()
       loss_list.append(loss.item())
       iter += 1
       
       
       if iter % 500 == 0 :
           correct = 0
           total = 0
           
           for images, labels in valid_loader :
               
               if torch.cuda.is_available() :
                   images = Variable(images.view(-1, seq_dim, input_dim).cuda())
               else :
                   images = Variable(images.view(-1, seq_dim, input_dim))
                   
               outputs = model(images)
               _, predicted = torch.max(outputs.data, 1)
               
               total += labels.size(0)
               if torch.cuda.is_available() : 
                   correct += (predicted.cpu() == labels.cpu()).sum()
                   
               else :
                   correct += (predicted == labels).sum()
                   
           accuracy = 100 * correct / total
           print(f'Iteration : {iter} Loss : {loss.item()} Accuracy : {accuracy}')
           

c:\Users\User\anaconda3\lib\site-packages\torch\nn\functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
c:\Users\User\anaconda3\lib\site-packages\torch\nn\functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration : 500 Loss : 2.1275174617767334 Accuracy : 26.6299991607666
Iteration : 1000 Loss : 0.6415965557098389 Accuracy : 78.63999938964844
Iteration : 1500 Loss : 0.4460902512073517 Accuracy : 88.19999694824219
Iteration : 2000 Loss : 0.1142461895942688 Accuracy : 92.20999908447266
Iteration : 2500 Loss : 0.11395461857318878 Accuracy : 95.66000366210938
Iteration : 3000 Loss : 0.07261805236339569 Accuracy : 95.54000091552734
Iteration : 3500 Loss : 0.11127766221761703 Accuracy : 96.19999694824219
Iteration : 4000 Loss : 0.07671014964580536 Accuracy : 97.19000244140625
Iteration : 4500 Loss : 0.10483045876026154 Accuracy : 96.95999908447266
Iteration : 5000 Loss : 0.014220238663256168 Accuracy : 97.3499984741211
Iteration : 5500 Loss : 0.019368406385183334 Accuracy : 97.36000061035156
Iteration : 6000 Loss : 0.17768767476081848 Accuracy : 97.68000030517578
Iteration : 6500 Loss : 0.010423594154417515 Accuracy : 97.87999725341797
Iteration : 7000 Loss : 0.06145305931568146 Accuracy : 

## 모델 학습 및 성능 체크

In [95]:
def evaluate(model, val_iter) :
    corrects, total, total_loss = 0, 0, 0
    model.eval()
    for images, labels in val_iter :
        if torch.cuda.is_available() :
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        else :
            images = Variable(images.view(-1, seq_dim, input_dim).to(device))
            
        logit = model(images).to(device)
        loss = F.cross_entropy(logit, labels, reduction='sum')
        _, predicted = torch.max(logit.data, 1)
        total += labels.size(0)
        total_loss += loss.item()
        corrects += (predicted == labels).sum()
        
    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

## 테스트 데이터셋 검증

In [96]:
test_loss, test_acc = evaluate(model, test_loader)
print(f'Test Loss : {test_loss:5.2f} | Test Accuracy : {test_acc:5.2f}')

Test Loss :  0.06 | Test Accuracy :  0.98


-----
# 실제 데이터 적용

In [98]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

data = pd.read_csv("C:/Users/User/github/Quant/data/kospi_category.csv", encoding='cp949')
start_date = '2004-01-01'
end_date = '2020-03-01'
 
data.index = pd.date_range(start_date,end_date,freq='m')

data.drop('date',axis=1, inplace=True)

In [100]:
data

,제조업,음식료품,섬유의복,종이목재,화학,의약품,비금속광물,철강금속,기계,전기전자,...,유통업,전기가스업,건설업,운수창고업,통신업,금융업,은행,증권,보험,서비스업
2004-01-31,1481.36,1301.41,87.67,181.95,1030.53,1181.26,850.49,1788.47,404.42,5205.42,...,224.08,604.60,63.13,972.18,335.00,263.87,191.01,1069.30,5300.97,355.96
2004-02-29,1534.92,1273.45,87.14,204.97,1116.18,1269.49,832.06,1963.28,439.56,5379.51,...,223.29,621.45,65.84,944.13,349.87,284.40,203.09,1121.27,5538.77,368.91
2004-03-31,1566.62,1258.50,82.87,206.01,1110.90,1241.57,807.16,1851.05,444.92,5685.00,...,216.23,611.06,65.94,888.41,325.26,270.62,187.51,1064.50,5818.82,389.92
2004-04-30,1523.14,1334.90,83.46,189.07,1139.42,1277.78,835.93,1674.74,400.92,5533.12,...,253.01,603.30,66.44,802.05,307.35,260.67,182.69,996.81,5895.44,417.70
2004-05-31,1400.71,1218.45,75.89,169.21,1091.50,1146.92,736.18,1586.80,359.56,5006.79,...,265.51,585.04,59.31,774.69,298.39,237.73,171.75,844.79,5455.42,400.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-31,4971.85,3410.15,282.12,338.47,4559.73,10792.64,1507.52,3508.32,698.16,18208.01,...,350.33,825.45,93.38,1280.08,339.66,390.70,246.57,1678.24,12886.06,1166.28
2019-11-30,4930.53,3431.14,287.74,330.44,4479.44,10229.68,1468.46,3605.86,673.26,18186.96,...,355.86,878.30,91.72,1298.96,348.64,401.50,245.59,1709.26,13464.64,1176.93
2019-12-31,5273.05,3480.12,289.14,330.11,4591.69,11031.00,1489.51,3693.15,701.59,20207.86,...,369.33,876.02,94.07,1330.05,345.69,409.49,246.62,1739.59,13698.26,1219.07
2020-01-31,5160.22,3295.05,259.04,352.95,4267.39,10902.30,1408.72,3395.63,649.76,20403.97,...,350.09,794.12,86.48,1250.76,329.30,372.14,224.73,1596.21,12249.88,1202.74


In [103]:
data.iloc[-10:, :]

,제조업,음식료품,섬유의복,종이목재,화학,의약품,비금속광물,철강금속,기계,전기전자,...,유통업,전기가스업,건설업,운수창고업,통신업,금융업,은행,증권,보험,서비스업
2019-05-31,4670.68,3645.13,292.80,381.55,4669.26,9951.76,1651.96,3969.08,737.95,15669.14,...,377.70,851.31,108.39,1447.12,358.04,427.92,289.39,1808.10,15579.44,1091.96
2019-06-30,4945.30,3740.90,298.43,391.83,4733.17,10349.47,1699.23,4145.27,765.30,17071.05,...,388.00,844.46,113.78,1363.13,368.47,438.51,293.51,1969.14,15468.15,1112.03
2019-07-31,4710.39,3450.11,285.99,343.65,4525.94,8937.99,1506.53,3815.12,716.77,16689.78,...,353.53,901.27,98.33,1283.44,350.47,407.35,274.70,1803.42,14388.27,1106.70
2019-08-31,4606.33,3251.38,266.49,364.67,4369.96,8538.84,1543.00,3638.72,709.21,16277.85,...,341.96,827.50,94.16,1290.76,342.16,379.80,259.04,1721.07,12879.06,1118.23
2019-09-30,4896.77,3453.54,285.42,348.74,4461.52,8978.69,1522.35,3778.30,744.99,17780.19,...,349.36,837.46,100.16,1265.60,347.23,391.95,275.50,1767.69,13150.57,1138.31
2019-10-31,4971.85,3410.15,282.12,338.47,4559.73,10792.64,1507.52,3508.32,698.16,18208.01,...,350.33,825.45,93.38,1280.08,339.66,390.70,246.57,1678.24,12886.06,1166.28
2019-11-30,4930.53,3431.14,287.74,330.44,4479.44,10229.68,1468.46,3605.86,673.26,18186.96,...,355.86,878.30,91.72,1298.96,348.64,401.50,245.59,1709.26,13464.64,1176.93
2019-12-31,5273.05,3480.12,289.14,330.11,4591.69,11031.00,1489.51,3693.15,701.59,20207.86,...,369.33,876.02,94.07,1330.05,345.69,409.49,246.62,1739.59,13698.26,1219.07
2020-01-31,5160.22,3295.05,259.04,352.95,4267.39,10902.30,1408.72,3395.63,649.76,20403.97,...,350.09,794.12,86.48,1250.76,329.30,372.14,224.73,1596.21,12249.88,1202.74
2020-02-29,4893.83,3178.68,232.81,320.97,4037.85,10565.78,1340.51,3147.69,608.82,19573.08,...,324.40,678.05,78.51,1226.31,310.52,334.07,198.91,1467.80,11085.54,1129.52


In [144]:
X = data.iloc[:-26, :]
y = data.iloc[-26:, :]
print(X)
print(y)

                제조업     음식료품    섬유의복    종이목재       화학       의약품    비금속광물  \
2004-01-31  1481.36  1301.41   87.67  181.95  1030.53   1181.26   850.49   
2004-02-29  1534.92  1273.45   87.14  204.97  1116.18   1269.49   832.06   
2004-03-31  1566.62  1258.50   82.87  206.01  1110.90   1241.57   807.16   
2004-04-30  1523.14  1334.90   83.46  189.07  1139.42   1277.78   835.93   
2004-05-31  1400.71  1218.45   75.89  169.21  1091.50   1146.92   736.18   
...             ...      ...     ...     ...      ...       ...      ...   
2017-08-31  5315.28  4428.05  296.83  327.21  5842.06   9841.25  1299.77   
2017-09-30  5515.26  4240.81  281.13  310.72  5698.48  10693.90  1185.46   
2017-10-31  5837.07  4431.02  287.26  308.18  5994.89  11528.94  1294.92   
2017-11-30  5687.52  4441.53  306.32  309.13  6065.43  11656.16  1443.25   
2017-12-31  5648.76  4364.77  294.92  298.95  6071.54  12370.06  1352.51   

               철강금속      기계      전기전자  ...     유통업    전기가스업     건설업    운수창고업  \
2004-01

In [145]:
ms = MinMaxScaler()
ss = StandardScaler()

X_ss = ss.fit_transform(X)
y_ms = ms.fit_transform(y)


In [149]:

X_train = X_ss[:84, :]
X_test = X_ss[84:, :]

y_train = y_ms[:13, :]
y_test = y_ms[13:, :]

print('Training Shape :', X_train.shape, y_train.shape)
print('Testing Shape :', X_test.shape, y_test.shape)

Training Shape : (84, 22) (13, 22)
Testing Shape : (84, 22) (13, 22)


In [150]:
# 데이터셋 형태, 크기 조정
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

X_train_tensors_f = torch.reshape(X_train_tensors, (X_train_tensors.shape[0],
                                                  1, X_train_tensors.shape[1]))
X_test_tensors_f = torch.reshape(X_test_tensors, (X_test_tensors.shape[0],
                                                 1, X_test_tensors.shape[1]))

print('Training Shape :', X_train_tensors_f.shape, y_train_tensors.shape)
print('Testing Shape :', X_test_tensors_f.shape, y_test_tensors.shape)

Training Shape : torch.Size([84, 1, 22]) torch.Size([13, 22])
Testing Shape : torch.Size([84, 1, 22]) torch.Size([13, 22])


In [151]:
class LSTM(nn.Module) :
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length) :
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                           num_layers=num_layers, batch_first=True)
        self.fc_1 = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x) :
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        return out

In [152]:
# 변수값 설정
num_epochs = 1000
lr = 0.0001

input_size=22
hidden_size=2
num_layers=1

num_classes=1
model = LSTM(num_classes, input_size, hidden_size, num_layers, X_train_tensors_f.shape[1])

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [153]:
for epoch in range(num_epochs) :
    outputs = model.forward(X_train_tensors_f)
    optimizer.zero_grad()
    loss = criterion(outputs, y_train_tensors)
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0 :
        print(f'Epoch : {epoch}, loss : {loss.item():1.5f}')

c:\Users\User\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([13, 22])) that is different to the input size (torch.Size([84, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (84) must match the size of tensor b (13) at non-singleton dimension 0

: 

In [ ]:
df_x_ss = ss.transform(data.iloc[:, :-1])
df_y_ms = ms.transform(data.iloc[:, -1:])

df_x_ss = Variable(torch.Tensor(df_x_ss))
df_y_ms = Variable(torch.Tensor(df_y_ms))
df_x_ss = torch.reshape(df_x_ss, (df_x_ss.shape[0], 1, df_x_ss.shape[1]))
train_predict = model(df_x_ss)
predicted = train_predict.data.numpy()
label_y = df_y_ms.data.numpy()

predicted = ms.inverse_transform(predicted)
label_y = ms.inverse_transform(label_y)
plt.figure(figsize=(10, 6))
plt.axvline(x=200, c='r', linestyle='--')

plt.plot(label_y, label='Actual Data')
plt.plot(predicted, label='Predicted Data')
plt.title('Time-Series Prediction')
plt.legend()
plt.show()